In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from PIL import Image
from skimage.transform import resize
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_y')
test_fov = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_fov')
train_x, train_y, test_x, test_y, test_fov

(dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512), dtype=float32, chunksize=(20, 512, 512)>)

In [3]:
def get_inside_fov(test_fov):
    flat = test_fov.ravel()
    flat = da.where(flat>0.5, 1, 0)
    index_inside_fov = da.where(flat)[0]
    return index_inside_fov
inside_fov = get_inside_fov(test_fov).compute()
inside_fov

array([  14069,   14070,   14071, ..., 5235971, 5235972, 5235973])

In [4]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters=32, dropout=0.1):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    #drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(pool_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    #drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(pool_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4) #(-1, 8, 8, 128)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_6) #(-1, 16, 16, 128)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 16, 16, 192)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 16, 16, 64)
    conv_16 = build_block(conv_15, n_filters * 2)
    #drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_16) #(-1, 32, 32, 64)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 32, 32, 92)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 32, 32, 32)
    conv_18 = build_block(conv_17, n_filters)
    #drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(conv_18)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

def random_crop(img, msk, random_crop_size):
    # Note: image_data_format is 'channel_last'
    #assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :], msk[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_x_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        batch_y_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        for i in range(batch_x.shape[0]):
            batch_x_crops[i], batch_y_crops[i] = random_crop(batch_x[i], batch_y[i], (crop_length, crop_length))
        yield (batch_x_crops, batch_y_crops)
        
class AUCTensorBoard(tf.keras.callbacks.TensorBoard):
    '''
    Add learning rate evolution to Tensorboard
    '''
    def __init__(self, log_dir, training_data, validation_data):
        super().__init__(log_dir=log_dir)
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs=None):
        bin_test_y = np.where(self.y_val.compute()>0.5, 1, 0)
        y_pred_val = self.model.predict(self.x_val, batch_size=1)
        roc_val = roc_auc_score(bin_test_y.ravel()[inside_fov], y_pred_val.ravel()[inside_fov])
        print('aucroc = ', roc_val)
        logs['auc'] = roc_val
        super().on_epoch_end(epoch, logs)
        
model = build_unet(input_shape=(None, None, 1))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 288         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [5]:
model.load_weights("day_4.weights.best.hdf5")
opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y, batch_size=1)
scores

20/20 [==============================] - 12s 587ms/step


[0.08612685240805149, 0.9224424362182617]

In [6]:
epochs, batch_size, lr, filepath = 1000000, 4, 0.001, "day_4.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True) #width_shift_range=0.1, height_shift_range=0.1, 
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = crop_generator(zip(image_generator, mask_generator), 32)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())
auc_roc = AUCTensorBoard(log_dir=os.getcwd(), training_data=(train_x, train_y),validation_data=(test_x, test_y))

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=17021,
                    callbacks=[checkpoint, auc_roc])

Epoch 17022/1000000
1/5 [=====>........................] - ETA: 15s - loss: 0.3749 - acc: 0.9175
Epoch 17022: val_acc improved from -inf to 0.92425, saving model to day_4.weights.best.hdf5
aucroc =  0.9786845948414779
5/5 [==============================] - 9s 2s/step - loss: 0.3502 - acc: 0.9217 - val_loss: 0.0839 - val_acc: 0.9243
Epoch 17023/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3428 - acc: 0.9171
Epoch 17023: val_acc improved from 0.92425 to 0.92441, saving model to day_4.weights.best.hdf5
aucroc =  0.9781105759237152
5/5 [==============================] - 4s 739ms/step - loss: 0.3282 - acc: 0.9228 - val_loss: 0.0841 - val_acc: 0.9244
Epoch 17024/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.1875 - acc: 0.9580
Epoch 17024: val_acc did not improve from 0.92441
aucroc =  0.9768131133942268
5/5 [==============================] - 4s 733ms/step - loss: 0.3898 - acc: 0.9187 - val_loss: 0.0863 - val_acc: 0.9240
Epoch 17025/1000000
1/5 [=====>..

Epoch 17050/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4248 - acc: 0.9070
Epoch 17050: val_acc did not improve from 0.92441
aucroc =  0.976579552606327
5/5 [==============================] - 4s 737ms/step - loss: 0.3979 - acc: 0.9138 - val_loss: 0.0862 - val_acc: 0.9237
Epoch 17051/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.3593 - acc: 0.9177
Epoch 17051: val_acc did not improve from 0.92441
aucroc =  0.9765698050802806
5/5 [==============================] - 4s 769ms/step - loss: 0.4073 - acc: 0.9153 - val_loss: 0.0864 - val_acc: 0.9239
Epoch 17052/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.4817 - acc: 0.8811
Epoch 17052: val_acc did not improve from 0.92441
aucroc =  0.9762026689371349
5/5 [==============================] - 4s 725ms/step - loss: 0.3402 - acc: 0.9266 - val_loss: 0.0875 - val_acc: 0.9239
Epoch 17053/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.3166 - acc: 0.9299
Epoch 17053: val_acc di

Epoch 17078/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.4850 - acc: 0.8914
Epoch 17078: val_acc did not improve from 0.92444
aucroc =  0.9761922030002398
5/5 [==============================] - 4s 746ms/step - loss: 0.3141 - acc: 0.9293 - val_loss: 0.0870 - val_acc: 0.9236
Epoch 17079/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5135 - acc: 0.8901
Epoch 17079: val_acc did not improve from 0.92444
aucroc =  0.9756952412313602
5/5 [==============================] - 4s 758ms/step - loss: 0.4927 - acc: 0.8968 - val_loss: 0.0870 - val_acc: 0.9239
Epoch 17080/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3946 - acc: 0.9145
Epoch 17080: val_acc did not improve from 0.92444
aucroc =  0.9755295930230456
5/5 [==============================] - 4s 752ms/step - loss: 0.3842 - acc: 0.9165 - val_loss: 0.0871 - val_acc: 0.9240
Epoch 17081/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3942 - acc: 0.9149
Epoch 17081: val_acc d

Epoch 17106/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.1946 - acc: 0.9502
Epoch 17106: val_acc did not improve from 0.92444
aucroc =  0.9777925096829602
5/5 [==============================] - 4s 768ms/step - loss: 0.2478 - acc: 0.9520 - val_loss: 0.0851 - val_acc: 0.9241
Epoch 17107/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3125 - acc: 0.9239
Epoch 17107: val_acc did not improve from 0.92444
aucroc =  0.9764395554685058
5/5 [==============================] - 4s 728ms/step - loss: 0.2895 - acc: 0.9306 - val_loss: 0.0860 - val_acc: 0.9242
Epoch 17108/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.2905 - acc: 0.9304
Epoch 17108: val_acc did not improve from 0.92444
aucroc =  0.9725046856993085
5/5 [==============================] - 4s 737ms/step - loss: 0.3014 - acc: 0.9288 - val_loss: 0.0897 - val_acc: 0.9238
Epoch 17109/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.3171 - acc: 0.9214
Epoch 17109: val_acc d

Epoch 17134/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.3231 - acc: 0.9353
Epoch 17134: val_acc did not improve from 0.92444
aucroc =  0.9765366255086252
5/5 [==============================] - 4s 741ms/step - loss: 0.3790 - acc: 0.9144 - val_loss: 0.0877 - val_acc: 0.9238
Epoch 17135/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.2737 - acc: 0.9331
Epoch 17135: val_acc did not improve from 0.92444
aucroc =  0.975680928778745
5/5 [==============================] - 4s 738ms/step - loss: 0.4093 - acc: 0.9094 - val_loss: 0.0887 - val_acc: 0.9237
Epoch 17136/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4034 - acc: 0.9267
Epoch 17136: val_acc did not improve from 0.92444
aucroc =  0.9759486376763434
5/5 [==============================] - 4s 741ms/step - loss: 0.3599 - acc: 0.9348 - val_loss: 0.0879 - val_acc: 0.9239
Epoch 17137/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5003 - acc: 0.8915
Epoch 17137: val_acc di

...

Epoch 24162/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.5186 - acc: 0.8699
Epoch 24162: val_acc did not improve from 0.92455
aucroc =  0.9769061645771434
5/5 [==============================] - 4s 705ms/step - loss: 0.3767 - acc: 0.9212 - val_loss: 0.0883 - val_acc: 0.9239
Epoch 24163/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.5858 - acc: 0.8611
Epoch 24163: val_acc did not improve from 0.92455
aucroc =  0.9765273377166337
5/5 [==============================] - 4s 706ms/step - loss: 0.4606 - acc: 0.8937 - val_loss: 0.0888 - val_acc: 0.9239
Epoch 24164/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.1874 - acc: 0.9480
Epoch 24164: val_acc did not improve from 0.92455
aucroc =  0.9764138899700664
5/5 [==============================] - 4s 718ms/step - loss: 0.4050 - acc: 0.9069 - val_loss: 0.0887 - val_acc: 0.9239
Epoch 24165/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4799 - acc: 0.8939
Epoch 24165: val_acc d

Epoch 24190/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.3339 - acc: 0.9338
Epoch 24190: val_acc did not improve from 0.92455
aucroc =  0.9781297533383057
5/5 [==============================] - 4s 726ms/step - loss: 0.4764 - acc: 0.9023 - val_loss: 0.0855 - val_acc: 0.9236
Epoch 24191/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3470 - acc: 0.9311
Epoch 24191: val_acc did not improve from 0.92455
aucroc =  0.9778170472589317
5/5 [==============================] - 4s 745ms/step - loss: 0.3066 - acc: 0.9372 - val_loss: 0.0856 - val_acc: 0.9236
Epoch 24192/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.4737 - acc: 0.9082
Epoch 24192: val_acc did not improve from 0.92455
aucroc =  0.9773880738579799
5/5 [==============================] - 4s 704ms/step - loss: 0.5546 - acc: 0.8833 - val_loss: 0.0854 - val_acc: 0.9238
Epoch 24193/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.5096 - acc: 0.8889
Epoch 24193: val_acc d

Epoch 24218/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.4914 - acc: 0.9119
Epoch 24218: val_acc did not improve from 0.92455
aucroc =  0.9775557122802339
5/5 [==============================] - 4s 742ms/step - loss: 0.4223 - acc: 0.9092 - val_loss: 0.0859 - val_acc: 0.9238
Epoch 24219/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.4229 - acc: 0.9053
Epoch 24219: val_acc did not improve from 0.92455
aucroc =  0.9769722732551043
5/5 [==============================] - 4s 766ms/step - loss: 0.4047 - acc: 0.9071 - val_loss: 0.0872 - val_acc: 0.9235
Epoch 24220/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.3814 - acc: 0.9265
Epoch 24220: val_acc did not improve from 0.92455
aucroc =  0.9765242036123732
5/5 [==============================] - 4s 714ms/step - loss: 0.4362 - acc: 0.9055 - val_loss: 0.0888 - val_acc: 0.9230
Epoch 24221/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.2598 - acc: 0.9348
Epoch 24221: val_acc d

Epoch 24246/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.4138 - acc: 0.9082
Epoch 24246: val_acc did not improve from 0.92455
aucroc =  0.9778265555286865
5/5 [==============================] - 4s 723ms/step - loss: 0.6124 - acc: 0.8864 - val_loss: 0.0854 - val_acc: 0.9242
Epoch 24247/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.1994 - acc: 0.9619
Epoch 24247: val_acc did not improve from 0.92455
aucroc =  0.9714479152675523
5/5 [==============================] - 4s 708ms/step - loss: 0.3569 - acc: 0.9198 - val_loss: 0.0995 - val_acc: 0.9229
Epoch 24248/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.2124 - acc: 0.9565
Epoch 24248: val_acc did not improve from 0.92455
aucroc =  0.9670847197322341
5/5 [==============================] - 4s 713ms/step - loss: 0.3898 - acc: 0.9118 - val_loss: 0.1071 - val_acc: 0.9219
Epoch 24249/1000000
1/5 [=====>........................] - ETA: 0s - loss: 0.6162 - acc: 0.8657
Epoch 24249: val_acc d

KeyboardInterrupt: 